### Import Dataset

test set are 2019 tournament data 

#### current best log-loss: 0.5210425754478005


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
import catboost as cb
import lightgbm as lgb

In [2]:
dftrain = pd.read_csv("X_train.csv")
dfeval = pd.read_csv("X_test.csv")

y_train = pd.read_csv("y_train.csv")
y_eval = pd.read_csv("y_test.csv")

In [3]:
print(dftrain)

      Unnamed: 0  team1_score WLoc  num_ot  team1_seed  strongseed  \
0              0           52    N       0           6           6   
1              1           61    N       0          15           2   
2              2           66    N       0          16          16   
3              3           64    N       0           9           8   
4              4           65    N       0           7           2   
...          ...          ...  ...     ...         ...         ...   
1241        1241           84    N       0           5           4   
1242        1242           50    N       0           9           9   
1243        1243           69    N       0           2           2   
1244        1244           95    N       0           1           1   
1245        1245           69    N       0           3           3   

     team2_region team1_region  season         host  ...  team1_adjtempo  \
0               X            X    2018      detroit  ...         68.7000   
1      

In [4]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 76 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      1246 non-null   int64  
 1   team1_score                     1246 non-null   int64  
 2   WLoc                            1246 non-null   object 
 3   num_ot                          1246 non-null   int64  
 4   team1_seed                      1246 non-null   int64  
 5   strongseed                      1246 non-null   int64  
 6   team2_region                    1246 non-null   object 
 7   team1_region                    1246 non-null   object 
 8   season                          1246 non-null   int64  
 9   host                            1246 non-null   object 
 10  host_lat                        1246 non-null   float64
 11  host_long                       1246 non-null   float64
 12  team2_lat                       12

In [5]:
features = ["sead_diff", "team1_seed","team1_adjoe","team2_adjoe","team1_adjde","team2_adjde","team1_blockpct","team2_blockpct",
           "team1_pt_team_season_wins","team2_pt_team_season_wins",
           "team1_pt_overall_s16","team2_pt_overall_s16","team1_pt_coach_season_wins","team2_pt_coach_season_wins",
           "team1_pt_school_ncaa","team2_pt_school_ncaa"]

In [6]:
dftrain = dftrain[features]

dfeval = dfeval[features]

In [7]:
y_train = y_train.drop(y_train.columns[0],axis=1)

In [8]:
y_eval = y_eval.drop(y_eval.columns[0],axis=1)

### LightGBM 

#### log-loss: 0.5210425754478005

梯度提升决策树模型

在梯度提升中，每个弱学习器都在上一个学习器的错误上进行训练，并专注于解决上一个学习器未能解决的问题。每个学习器的输出结果都会被结合到最终的决策中，从而提高预测的准确性。

容易受到异常值的影响 但是处理速度更快 

'n_estimators'是LightGBM中的一个参数，它控制了模型的弱学习器数量。弱学习器通常是决策树，它们的结果将被结合在一起来预测结果。设置'n_estimators'的值越大，模型的准确率也就越高，但同时也会增加模型的过拟合风险。因此，调整'n_estimators'的值需要权衡模型的准确率和泛化能力。一般来说，'n_estimators'的值在100到1000之间是比较合适的。

'subsample'是LightGBM中的一个参数，它控制了每次迭代用于训练的样本数量。它的值越大，模型的训练速度也就越快，但同时也会增加模型的过拟合风险。因此，调整'subsample'的值需要权衡模型的准确率和训练速度。一般来说，'subsample'的值在0.5到1.0之间是比较合适的。

'min_child_weight'是LightGBM中的一个参数，它用于控制决策树的生长。当决策树到达叶子节点时，它检查当前节点的样本权值和是否小于'min_child_weight'，如果是，则停止生长。'min_child_weight'的值越大，模型的准确率也可能越高，但同时也会增加模型的过拟合风险。因此，调整'min_child_weight'的值需要权衡模型的准确率和泛化能力。一般来说，'min_child_weight'的值在1到10之间是比较合适的。

'min_child_samples'是LightGBM中的一个参数，它用于控制决策树的生长。当决策树到达叶子节点时，它检查当前节点的样本数是否小于'min_child_samples'，如果是，则停止生长。'min_child_samples'的值越大，模型的准确率也可能越高，但同时也会增加模型的过拟合风险。因此，调整'min_child_samples'的值需要权衡模型的准确率和泛化能力。一般来说，'min_child_samples'的值在20到100之间是比较合适的。

'num_leaves'是LightGBM中的一个参数，它用于控制每棵树的叶子节点数。它的值越大，决策树的复杂度也就越高，模型的准确率也可能越高。但同时也会增加模型的过拟合风险。因此，调整'num_leaves'的值需要权衡模型的准确率和泛化能力。一般来说，'num_leaves'的值在31到255之间是比较合适的。

In [9]:
# Functions to print best score and paramters for baseline models
def print_best_score(gridsearch,param_test):
    print("The best score is: ", gridsearch.best_score_)
    print("The best param set is: ")
    best_parameters = gridsearch.best_parameters_.get_params()
    for param_name in sorted(param_test.keys()):
        print(param_name,best_parameters[param_name])

In [10]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 255), 
             'min_child_samples': sp_randint(20, 100), 
             'min_child_weight': sp_randint(1, 10),
             'subsample': sp_uniform(loc=0.5, scale=0.5), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [11]:
clf = lgb.LGBMClassifier(max_depth= 100, random_state=1,n_jobs=4, n_estimators=1000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=100,
    scoring='neg_log_loss',
    cv=3,
    random_state=1)

In [12]:
gs.fit(dftrain, y_train.values.ravel())

RandomizedSearchCV(cv=3,
                   estimator=LGBMClassifier(max_depth=100, n_estimators=1000,
                                            n_jobs=4, random_state=1),
                   n_iter=100,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020212F98A90>,
                                        'min_child_samples': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020212F91070>,
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020212F98610>,
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020212F94100>,
                                        'reg_alpha': [0, 0.1, 1, 2, 5, 7, 10,
                                                      50, 100],
                                        'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50,
                                 

In [60]:
y_pred=gs.predict_proba(dfeval)[:,1]

In [61]:
print("log_loss is: ", metrics.log_loss(y_eval,y_pred))
print("roc_auc is: ", metrics.roc_auc_score(y_eval,y_pred))

log_loss is:  0.5210425754478005
roc_auc is:  0.8223214285714285


### Random Forest
#### log-loss: 0.5460616776039692

In [70]:
maximum_depth = 100
minimum_depth = 1
param_grid = {'max_depth': list(range(minimum_depth, maximum_depth+1))}

# The reason we choose RandomizedSearchCV instead of GridSearchCV because it's faster
# The Random Forest Model
gs_rf = GridSearchCV(
    RandomForestClassifier(criterion='entropy', random_state=1), 
    param_grid,cv=3, n_jobs=4)

In [74]:
gs_rf.fit(dftrain, y_train.values.ravel())

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(criterion='entropy',
                                              random_state=1),
             n_jobs=4,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...]})

In [75]:
y_pred=gs_rf.predict_proba(dfeval)[:,1]

In [76]:
print("log_loss is: ", metrics.log_loss(y_eval,y_pred))
print("roc_auc is: ", metrics.roc_auc_score(y_eval,y_pred))

log_loss is:  0.5460616776039692
roc_auc is:  0.8133928571428571


### Catboost

#### log_loss is:  0.5377091842712927

In [27]:
import catboost as cb

param_distributions = {'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
                       'depth': [3, 5, 7, 9],
                       'l2_leaf_reg': [1, 3, 5, 7, 9]}

cb = cb.CatBoostClassifier(random_state=1,iterations=30)
rs_cb = RandomizedSearchCV(cb, param_distributions, n_iter=100, cv=5, scoring='neg_log_loss')


In [28]:
rs_cb.fit(dftrain, y_train.values)

0:	learn: 0.6911081	total: 622us	remaining: 18.1ms
1:	learn: 0.6890563	total: 1.15ms	remaining: 16.1ms
2:	learn: 0.6870613	total: 1.64ms	remaining: 14.8ms
3:	learn: 0.6851114	total: 2.12ms	remaining: 13.8ms
4:	learn: 0.6832915	total: 2.58ms	remaining: 12.9ms
5:	learn: 0.6811802	total: 3.05ms	remaining: 12.2ms
6:	learn: 0.6794728	total: 3.5ms	remaining: 11.5ms
7:	learn: 0.6775061	total: 3.99ms	remaining: 11ms
8:	learn: 0.6755598	total: 4.59ms	remaining: 10.7ms
9:	learn: 0.6738855	total: 5.14ms	remaining: 10.3ms
10:	learn: 0.6721978	total: 5.66ms	remaining: 9.78ms
11:	learn: 0.6704135	total: 6.2ms	remaining: 9.3ms
12:	learn: 0.6688946	total: 6.67ms	remaining: 8.73ms
13:	learn: 0.6671865	total: 7.17ms	remaining: 8.19ms
14:	learn: 0.6658856	total: 7.65ms	remaining: 7.65ms
15:	learn: 0.6641796	total: 8.19ms	remaining: 7.17ms
16:	learn: 0.6626896	total: 8.81ms	remaining: 6.74ms
17:	learn: 0.6610299	total: 9.33ms	remaining: 6.22ms
18:	learn: 0.6593054	total: 9.87ms	remaining: 5.71ms
19:	learn

1:	learn: 0.6704510	total: 1.27ms	remaining: 17.8ms
2:	learn: 0.6618915	total: 1.92ms	remaining: 17.3ms
3:	learn: 0.6535072	total: 2.44ms	remaining: 15.9ms
4:	learn: 0.6459537	total: 3ms	remaining: 15ms
5:	learn: 0.6381766	total: 3.5ms	remaining: 14ms
6:	learn: 0.6317722	total: 3.99ms	remaining: 13.1ms
7:	learn: 0.6253599	total: 4.52ms	remaining: 12.4ms
8:	learn: 0.6187663	total: 5.03ms	remaining: 11.7ms
9:	learn: 0.6123868	total: 5.62ms	remaining: 11.2ms
10:	learn: 0.6080644	total: 6.2ms	remaining: 10.7ms
11:	learn: 0.6029039	total: 6.71ms	remaining: 10.1ms
12:	learn: 0.5988678	total: 7.28ms	remaining: 9.52ms
13:	learn: 0.5949802	total: 7.84ms	remaining: 8.96ms
14:	learn: 0.5923021	total: 8.39ms	remaining: 8.39ms
15:	learn: 0.5882210	total: 8.96ms	remaining: 7.84ms
16:	learn: 0.5842361	total: 9.54ms	remaining: 7.3ms
17:	learn: 0.5799726	total: 10.1ms	remaining: 6.73ms
18:	learn: 0.5774912	total: 10.7ms	remaining: 6.18ms
19:	learn: 0.5743796	total: 11.2ms	remaining: 5.59ms
20:	learn: 0

1:	learn: 0.5826232	total: 1.34ms	remaining: 18.7ms
2:	learn: 0.5575614	total: 1.88ms	remaining: 16.9ms
3:	learn: 0.5490609	total: 2.42ms	remaining: 15.8ms
4:	learn: 0.5410642	total: 3.01ms	remaining: 15.1ms
5:	learn: 0.5321685	total: 3.6ms	remaining: 14.4ms
6:	learn: 0.5261528	total: 4.21ms	remaining: 13.8ms
7:	learn: 0.5238860	total: 4.71ms	remaining: 13ms
8:	learn: 0.5175281	total: 5.23ms	remaining: 12.2ms
9:	learn: 0.5154137	total: 5.74ms	remaining: 11.5ms
10:	learn: 0.5118525	total: 6.27ms	remaining: 10.8ms
11:	learn: 0.5086891	total: 6.78ms	remaining: 10.2ms
12:	learn: 0.5049597	total: 7.29ms	remaining: 9.53ms
13:	learn: 0.5004035	total: 7.87ms	remaining: 8.99ms
14:	learn: 0.4946974	total: 8.41ms	remaining: 8.41ms
15:	learn: 0.4884975	total: 8.93ms	remaining: 7.81ms
16:	learn: 0.4832134	total: 9.52ms	remaining: 7.28ms
17:	learn: 0.4792657	total: 10.1ms	remaining: 6.72ms
18:	learn: 0.4744702	total: 10.6ms	remaining: 6.13ms
19:	learn: 0.4685238	total: 11.2ms	remaining: 5.58ms
20:	l

1:	learn: 0.5627021	total: 1.59ms	remaining: 22.3ms
2:	learn: 0.5357878	total: 2.22ms	remaining: 20ms
3:	learn: 0.5289713	total: 2.82ms	remaining: 18.3ms
4:	learn: 0.5251952	total: 3.52ms	remaining: 17.6ms
5:	learn: 0.5188753	total: 4.21ms	remaining: 16.8ms
6:	learn: 0.5165799	total: 4.79ms	remaining: 15.7ms
7:	learn: 0.5045496	total: 5.37ms	remaining: 14.8ms
8:	learn: 0.4953260	total: 5.94ms	remaining: 13.9ms
9:	learn: 0.4871236	total: 6.66ms	remaining: 13.3ms
10:	learn: 0.4746773	total: 7.29ms	remaining: 12.6ms
11:	learn: 0.4666283	total: 7.95ms	remaining: 11.9ms
12:	learn: 0.4624610	total: 8.57ms	remaining: 11.2ms
13:	learn: 0.4515243	total: 9.26ms	remaining: 10.6ms
14:	learn: 0.4440228	total: 9.89ms	remaining: 9.89ms
15:	learn: 0.4360108	total: 10.7ms	remaining: 9.37ms
16:	learn: 0.4329744	total: 11.3ms	remaining: 8.66ms
17:	learn: 0.4213716	total: 12ms	remaining: 7.98ms
18:	learn: 0.4090373	total: 12.6ms	remaining: 7.32ms
19:	learn: 0.4055047	total: 13.2ms	remaining: 6.6ms
20:	lea

1:	learn: 0.6751191	total: 1.27ms	remaining: 17.7ms
2:	learn: 0.6683134	total: 1.84ms	remaining: 16.5ms
3:	learn: 0.6619521	total: 2.31ms	remaining: 15ms
4:	learn: 0.6555809	total: 2.87ms	remaining: 14.4ms
5:	learn: 0.6484709	total: 3.38ms	remaining: 13.5ms
6:	learn: 0.6429134	total: 3.89ms	remaining: 12.8ms
7:	learn: 0.6378625	total: 4.43ms	remaining: 12.2ms
8:	learn: 0.6322030	total: 4.95ms	remaining: 11.5ms
9:	learn: 0.6277758	total: 5.45ms	remaining: 10.9ms
10:	learn: 0.6238924	total: 5.91ms	remaining: 10.2ms
11:	learn: 0.6193122	total: 6.39ms	remaining: 9.59ms
12:	learn: 0.6156580	total: 6.87ms	remaining: 8.98ms
13:	learn: 0.6126892	total: 7.31ms	remaining: 8.35ms
14:	learn: 0.6100450	total: 7.76ms	remaining: 7.76ms
15:	learn: 0.6067306	total: 8.22ms	remaining: 7.2ms
16:	learn: 0.6038062	total: 8.67ms	remaining: 6.63ms
17:	learn: 0.6006205	total: 9.2ms	remaining: 6.13ms
18:	learn: 0.5974073	total: 9.71ms	remaining: 5.62ms
19:	learn: 0.5949618	total: 10.3ms	remaining: 5.15ms
20:	le

...

1:	learn: 0.6809922	total: 12.1ms	remaining: 169ms
2:	learn: 0.6760699	total: 17.6ms	remaining: 159ms
3:	learn: 0.6700869	total: 20.7ms	remaining: 134ms
4:	learn: 0.6664396	total: 27.2ms	remaining: 136ms
5:	learn: 0.6626885	total: 33ms	remaining: 132ms
6:	learn: 0.6589183	total: 38.9ms	remaining: 128ms
7:	learn: 0.6555388	total: 45.3ms	remaining: 125ms
8:	learn: 0.6511610	total: 51.4ms	remaining: 120ms
9:	learn: 0.6471807	total: 57.4ms	remaining: 115ms
10:	learn: 0.6422263	total: 63.3ms	remaining: 109ms
11:	learn: 0.6381259	total: 69.4ms	remaining: 104ms
12:	learn: 0.6348485	total: 75.8ms	remaining: 99.1ms
13:	learn: 0.6301067	total: 82.1ms	remaining: 93.8ms
14:	learn: 0.6263622	total: 89.2ms	remaining: 89.2ms
15:	learn: 0.6234741	total: 95.8ms	remaining: 83.8ms
16:	learn: 0.6206628	total: 103ms	remaining: 78.7ms
17:	learn: 0.6170476	total: 110ms	remaining: 73.1ms
18:	learn: 0.6138392	total: 116ms	remaining: 67.3ms
19:	learn: 0.6102839	total: 125ms	remaining: 62.3ms
20:	learn: 0.607449

23:	learn: 0.5515057	total: 198ms	remaining: 49.5ms
24:	learn: 0.5481344	total: 208ms	remaining: 41.5ms
25:	learn: 0.5446450	total: 216ms	remaining: 33.2ms
26:	learn: 0.5418642	total: 224ms	remaining: 24.9ms
27:	learn: 0.5394061	total: 230ms	remaining: 16.4ms
28:	learn: 0.5360240	total: 236ms	remaining: 8.14ms
29:	learn: 0.5330669	total: 242ms	remaining: 0us
0:	learn: 0.6797208	total: 7.07ms	remaining: 205ms
1:	learn: 0.6686858	total: 13ms	remaining: 182ms
2:	learn: 0.6573445	total: 14.3ms	remaining: 129ms
3:	learn: 0.6488803	total: 20.2ms	remaining: 131ms
4:	learn: 0.6414487	total: 26.6ms	remaining: 133ms
5:	learn: 0.6360794	total: 33.1ms	remaining: 132ms
6:	learn: 0.6265989	total: 39.4ms	remaining: 129ms
7:	learn: 0.6204029	total: 45.6ms	remaining: 125ms
8:	learn: 0.6152438	total: 52.3ms	remaining: 122ms
9:	learn: 0.6085603	total: 58.9ms	remaining: 118ms
10:	learn: 0.6030470	total: 65.1ms	remaining: 112ms
11:	learn: 0.5971689	total: 71ms	remaining: 107ms
12:	learn: 0.5919725	total: 7

9:	learn: 0.4930016	total: 49.7ms	remaining: 99.3ms
10:	learn: 0.4839534	total: 55.4ms	remaining: 95.8ms
11:	learn: 0.4754974	total: 61.3ms	remaining: 92ms
12:	learn: 0.4672393	total: 66.8ms	remaining: 87.4ms
13:	learn: 0.4586179	total: 72.3ms	remaining: 82.7ms
14:	learn: 0.4500939	total: 77.7ms	remaining: 77.7ms
15:	learn: 0.4432933	total: 83ms	remaining: 72.6ms
16:	learn: 0.4364653	total: 88.2ms	remaining: 67.5ms
17:	learn: 0.4284408	total: 93.9ms	remaining: 62.6ms
18:	learn: 0.4241416	total: 99.2ms	remaining: 57.5ms
19:	learn: 0.4184339	total: 105ms	remaining: 52.4ms
20:	learn: 0.4114942	total: 110ms	remaining: 47.1ms
21:	learn: 0.4048085	total: 115ms	remaining: 41.8ms
22:	learn: 0.3972504	total: 121ms	remaining: 36.8ms
23:	learn: 0.3910188	total: 127ms	remaining: 31.8ms
24:	learn: 0.3859972	total: 132ms	remaining: 26.5ms
25:	learn: 0.3809442	total: 139ms	remaining: 21.4ms
26:	learn: 0.3775094	total: 146ms	remaining: 16.2ms
27:	learn: 0.3706908	total: 152ms	remaining: 10.8ms
28:	lea

19:	learn: 0.3357037	total: 121ms	remaining: 60.5ms
20:	learn: 0.3350178	total: 127ms	remaining: 54.5ms
21:	learn: 0.3253878	total: 134ms	remaining: 48.8ms
22:	learn: 0.3179753	total: 140ms	remaining: 42.8ms
23:	learn: 0.3175114	total: 147ms	remaining: 36.7ms
24:	learn: 0.3064626	total: 153ms	remaining: 30.6ms
25:	learn: 0.3048849	total: 160ms	remaining: 24.6ms
26:	learn: 0.2989883	total: 167ms	remaining: 18.5ms
27:	learn: 0.2943620	total: 174ms	remaining: 12.4ms
28:	learn: 0.2893055	total: 182ms	remaining: 6.28ms
29:	learn: 0.2856339	total: 189ms	remaining: 0us
0:	learn: 0.5897274	total: 7.45ms	remaining: 216ms
1:	learn: 0.5502323	total: 14.9ms	remaining: 208ms
2:	learn: 0.5235650	total: 21.5ms	remaining: 193ms
3:	learn: 0.5060886	total: 28.1ms	remaining: 183ms
4:	learn: 0.4866341	total: 34.2ms	remaining: 171ms
5:	learn: 0.4696190	total: 40.5ms	remaining: 162ms
6:	learn: 0.4553727	total: 46.3ms	remaining: 152ms
7:	learn: 0.4394920	total: 52ms	remaining: 143ms
8:	learn: 0.4181074	total

RandomizedSearchCV(cv=5,
                   estimator=<catboost.core.CatBoostClassifier object at 0x0000011DCC4F7F10>,
                   n_iter=100,
                   param_distributions={'depth': [3, 5, 7, 9],
                                        'l2_leaf_reg': [1, 3, 5, 7, 9],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.5,
                                                          1]},
                   scoring='neg_log_loss')

In [29]:
y_pred=rs_cb.predict_proba(dfeval)[:,1]

In [30]:
print("log_loss is: ", metrics.log_loss(y_eval,y_pred))
print("roc_auc is: ", metrics.roc_auc_score(y_eval,y_pred))

log_loss is:  0.5377091842712927
roc_auc is:  0.80625


### SVM 

In [81]:
from sklearn.svm import SVC
svm_model = SVC(probability=True)
hyperparameters = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': [0.1, 1, 'scale', 'auto']}

In [ ]:
svm_grid = GridSearchCV(svm_model, hyperparameters, cv=5)
svm_grid.fit(dftrain, y_train.values.ravel())

In [ ]:
y_pred=svm_grid.predict_proba(dfeval)[:,1]

In [ ]:
print("log_loss is: ", metrics.log_loss(y_eval,y_pred))
print("roc_auc is: ", metrics.roc_auc_score(y_eval,y_pred))

### Ensemble Model

### Bayesian Optimization

from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.linear_model import LogisticRegression

定义目标函数和超参数搜索空间
def lgbm_cv(num_leaves, learning_rate, n_estimators, max_depth):
    clf = lgb.LGBMClassifier(
        num_leaves = int(num_leaves),
        learning_rate = learning_rate,
        n_estimators = int(n_estimators),
        max_depth = int(max_depth)
    )
    return np.mean(cross_val_score(clf, dftrain, y_train.values.ravel(), cv=5, scoring="roc_auc"))

def rf_cv(n_estimators, max_depth, max_features):
    clf = RandomForestClassifier(
        n_estimators = int(n_estimators),
        max_depth = int(max_depth),
        max_features = max_features
    )
    return np.mean(cross_val_score(clf, dftrain, y_train.values.ravel(), cv=5, scoring="roc_auc"))

def cat_cv(depth, learning_rate, iterations):
    clf = cb.CatBoostClassifier(
        max_depth = int(depth),
        learning_rate = learning_rate,
        iterations = int(iterations),
        random_state = 42,
        logging_level = "Silent"
    )
    return np.mean(cross_val_score(clf, dftrain, y_train.values.ravel(), cv=5, scoring="roc_auc"))

lgbm_params = {
    "num_leaves": (10, 50),
    "learning_rate": (0.01, 0.1),
    "n_estimators": (50, 200),
    "max_depth": (3, 10)
}

rf_params = {
    "n_estimators": (50, 200),
    "max_depth": (3, 10),
    "max_features": (0.1, 0.9)
}

cat_params = {
    "depth": (3, 10),
    "learning_rate": (0.01, 0.1),
    "iterations": (50,200)
}

使用贝叶斯优化算法搜索最优超参数
lgbm_bo = BayesianOptimization(lgbm_cv, lgbm_params)
lgbm_bo.maximize(init_points=5, n_iter=15)

rf_bo = BayesianOptimization(rf_cv, rf_params)
rf_bo.maximize(init_points=5, n_iter=15)

cat_bo = BayesianOptimization(cat_cv, cat_params)
cat_bo.maximize(init_points=5, n_iter=15)

训练基分类器，并使用它们预测测试数据
lgbm_clf = lgb.LGBMClassifier(**lgbm_bo.max["params"])
lgbm_clf.fit(dftrain, y_train.values.ravel())
lgbm_pred = lgbm_clf.predict_proba(dfeval)[:, 1]

rf_clf = RandomForestClassifier(**rf_bo.max["params"])
rf_clf.fit(dftrain, y_train.values.ravel())
rf_pred = rf_clf.predict_proba(dfeval)[:, 1]

cat_clf = CatBoostClassifier(**cat_bo.max["params"], random_state=42, logging_level="Silent")
cat_clf.fit(dftrain, y_train.values.ravel())
cat_pred = cat_clf.predict_proba(dfeval)[:, 1]
